# Лабораторная работа 4

Tensorflow 2.x

1) Подготовка данных

2) Использование Keras Model API

3) Использование Keras Sequential + Functional API

https://www.tensorflow.org/tutorials

Для выполнения лабораторной работы необходимо установить tensorflow версии 2.0 или выше .

Рекомендуется использовать возможности Colab'а по обучению моделей на GPU.



In [1]:
import tensorflow as tf
import numpy as np
import os
import math
import timeit
import matplotlib.pyplot as plt

%matplotlib inline


USE_GPU = True

if USE_GPU:
    device = '/device:GPU:0'
else:
    device = '/cpu:0'

# Constant to control how often we print when training models.
print_every = 100
print('Using device: ', device)

Using device:  /device:GPU:0


# Подготовка данных
Загрузите набор данных из предыдущей лабораторной работы.

In [2]:
def load_mnist(num_training=49000, num_validation=1000, num_test=10000):
    """
    Fetch the CIFAR-10 dataset from the web and perform preprocessing to prepare
    it for the two-layer neural net classifier. These are the same steps as
    we used for the SVM, but condensed to a single function.
    """
    # Load the raw CIFAR-10 dataset and use appropriate data types and shapes
    mnist = tf.keras.datasets.mnist.load_data()
    (X_train, y_train), (X_test, y_test) = mnist
    X_train = np.asarray(X_train, dtype=np.float32)
    y_train = np.asarray(y_train, dtype=np.int32).flatten()
    X_test = np.asarray(X_test, dtype=np.float32)
    y_test = np.asarray(y_test, dtype=np.int32).flatten()

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Normalize the data: subtract the mean pixel and divide by std
    mean_pixel = X_train.mean(axis=(0, 1, 2), keepdims=True)
    std_pixel = X_train.std(axis=(0, 1, 2), keepdims=True)
    X_train = (X_train - mean_pixel) / std_pixel
    X_val = (X_val - mean_pixel) / std_pixel
    X_test = (X_test - mean_pixel) / std_pixel

    return X_train, y_train, X_val, y_val, X_test, y_test

# If there are errors with SSL downloading involving self-signed certificates,
# it may be that your Python version was recently installed on the current machine.
# See: https://github.com/tensorflow/tensorflow/issues/10779
# To fix, run the command: /Applications/Python\ 3.7/Install\ Certificates.command
#   ...replacing paths as necessary.

# Invoke the above function to get our data.
NHW = (0, 1, 2)
X_train, y_train, X_val, y_val, X_test, y_test = load_mnist()
X_train = X_train.reshape(-1, 28, 28, 1)
X_val = X_val.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Train data shape:  (49000, 28, 28, 1)
Train labels shape:  (49000,) int32
Validation data shape:  (1000, 28, 28, 1)
Validation labels shape:  (1000,)
Test data shape:  (10000, 28, 28, 1)
Test labels shape:  (10000,)


In [3]:
class Dataset(object):
    def __init__(self, X, y, batch_size, shuffle=False):
        """
        Construct a Dataset object to iterate over data X and labels y

        Inputs:
        - X: Numpy array of data, of any shape
        - y: Numpy array of labels, of any shape but with y.shape[0] == X.shape[0]
        - batch_size: Integer giving number of elements per minibatch
        - shuffle: (optional) Boolean, whether to shuffle the data on each epoch
        """
        assert X.shape[0] == y.shape[0], 'Got different numbers of data and labels'
        self.X, self.y = X, y
        self.batch_size, self.shuffle = batch_size, shuffle

    def __iter__(self):
        N, B = self.X.shape[0], self.batch_size
        idxs = np.arange(N)
        if self.shuffle:
            np.random.shuffle(idxs)
        return iter((self.X[i:i+B], self.y[i:i+B]) for i in range(0, N, B))


train_dset = Dataset(X_train, y_train, batch_size=64, shuffle=True)
val_dset = Dataset(X_val, y_val, batch_size=64, shuffle=False)
test_dset = Dataset(X_test, y_test, batch_size=64)

In [4]:
# We can iterate through a dataset like this:
for t, (x, y) in enumerate(train_dset):
    print(t, x.shape, y.shape)
    if t > 5: break

0 (64, 28, 28, 1) (64,)
1 (64, 28, 28, 1) (64,)
2 (64, 28, 28, 1) (64,)
3 (64, 28, 28, 1) (64,)
4 (64, 28, 28, 1) (64,)
5 (64, 28, 28, 1) (64,)
6 (64, 28, 28, 1) (64,)


#  Keras Model Subclassing API


Для реализации собственной модели с помощью Keras Model Subclassing API необходимо выполнить следующие шаги:

1) Определить новый класс, который является наследником tf.keras.Model.

2) В методе __init__() определить все необходимые слои из модуля tf.keras.layer

3) Реализовать прямой проход в методе call() на основе слоев, объявленных в __init__()

Ниже приведен пример использования keras API для определения двухслойной полносвязной сети.

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras

In [5]:
class TwoLayerFC(tf.keras.Model):
    def __init__(self, hidden_size, num_classes):
        super(TwoLayerFC, self).__init__()
        initializer = tf.initializers.VarianceScaling(scale=2.0)
        self.fc1 = tf.keras.layers.Dense(hidden_size, activation='relu',
                                   kernel_initializer=initializer)
        self.fc2 = tf.keras.layers.Dense(num_classes, activation='softmax',
                                   kernel_initializer=initializer)
        self.flatten = tf.keras.layers.Flatten()

    def call(self, x, training=False):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x


def test_TwoLayerFC():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    x = tf.zeros((64, input_size))
    model = TwoLayerFC(hidden_size, num_classes)
    # with tf.device(device):
    scores = model(x)
    print(scores.shape)

test_TwoLayerFC()

(64, 10)


D:\anakonda\lib\site-packages\keras\initializers\initializers_v2.py:120: UserWarning: The initializer VarianceScaling is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


Реализуйте трехслойную CNN для вашей задачи классификации.

Архитектура сети:
    
1. Сверточный слой (5 x 5 kernels, zero-padding = 'same')
2. Функция активации ReLU
3. Сверточный слой (3 x 3 kernels, zero-padding = 'same')
4. Функция активации ReLU
5. Полносвязный слой
6. Функция активации Softmax

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Conv2D

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dense

In [6]:
class ThreeLayerConvNet(tf.keras.Model):
    def __init__(self, channel_1, channel_2, num_classes):
        super(ThreeLayerConvNet, self).__init__()
        ########################################################################
        # Определение слоев для сверточной нейронной сети.
        ########################################################################
        self.conv1 = tf.keras.layers.Conv2D(channel_1, (5, 5), padding='same', activation='relu')
        self.conv2 = tf.keras.layers.Conv2D(channel_2, (3, 3), padding='same', activation='relu')
        self.flatten = tf.keras.layers.Flatten()
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, x, training=False):
        ########################################################################
        # Прямой проход для сверточной нейронной сети.
        ########################################################################
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x

In [7]:
def test_ThreeLayerConvNet():
    channel_1, channel_2, num_classes = 12, 8, 10
    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)
    with tf.device(device):
        x = tf.zeros((64, 3, 32, 32))
        scores = model(x)
        print(scores.shape)

test_ThreeLayerConvNet()

(64, 10)


Пример реализации процесса обучения:

In [8]:
def train_part34(model_init_fn, optimizer_init_fn, num_epochs=1, is_training=False):
    """
    Simple training loop for use with models defined using tf.keras. It trains
    a model for one epoch on the CIFAR-10 training set and periodically checks
    accuracy on the CIFAR-10 validation set.

    Inputs:
    - model_init_fn: A function that takes no parameters; when called it
      constructs the model we want to train: model = model_init_fn()
    - optimizer_init_fn: A function which takes no parameters; when called it
      constructs the Optimizer object we will use to optimize the model:
      optimizer = optimizer_init_fn()
    - num_epochs: The number of epochs to train for

    Returns: Nothing, but prints progress during trainingn
    """
    with tf.device(device):

        # Compute the loss like we did in Part II
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

        model = model_init_fn()
        optimizer = optimizer_init_fn()

        train_loss = tf.keras.metrics.Mean(name='train_loss')
        train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

        val_loss = tf.keras.metrics.Mean(name='val_loss')
        val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')

        t = 0
        for epoch in range(num_epochs):

            # Reset the metrics - https://www.tensorflow.org/alpha/guide/migration_guide#new-style_metrics
            train_loss.reset_states()
            train_accuracy.reset_states()

            for x_np, y_np in train_dset:
                with tf.GradientTape() as tape:

                    # Use the model function to build the forward pass.
                    scores = model(x_np, training=is_training)
                    loss = loss_fn(y_np, scores)

                    gradients = tape.gradient(loss, model.trainable_variables)
                    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

                    # Update the metrics
                    train_loss.update_state(loss)
                    train_accuracy.update_state(y_np, scores)

                    if t % print_every == 0:
                        val_loss.reset_states()
                        val_accuracy.reset_states()
                        for test_x, test_y in val_dset:
                            # During validation at end of epoch, training set to False
                            prediction = model(test_x, training=False)
                            t_loss = loss_fn(test_y, prediction)

                            val_loss.update_state(t_loss)
                            val_accuracy.update_state(test_y, prediction)

                        template = 'Iteration {}, Epoch {}, Loss: {}, Accuracy: {}, Val Loss: {}, Val Accuracy: {}'
                        print (template.format(t, epoch+1,
                                             train_loss.result(),
                                             train_accuracy.result()*100,
                                             val_loss.result(),
                                             val_accuracy.result()*100))
                    t += 1

In [9]:

hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn():
    return TwoLayerFC(hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.8840699195861816, Accuracy: 7.8125, Val Loss: 2.6917357444763184, Val Accuracy: 23.600000381469727
Iteration 100, Epoch 1, Loss: 0.6280143857002258, Accuracy: 81.09529876708984, Val Loss: 0.5261278748512268, Val Accuracy: 82.5
Iteration 200, Epoch 1, Loss: 0.5032378435134888, Accuracy: 85.29228210449219, Val Loss: 0.44090139865875244, Val Accuracy: 87.0
Iteration 300, Epoch 1, Loss: 0.44750645756721497, Accuracy: 86.92379760742188, Val Loss: 0.4048703610897064, Val Accuracy: 87.0999984741211
Iteration 400, Epoch 1, Loss: 0.4049050211906433, Accuracy: 88.16630554199219, Val Loss: 0.3590394854545593, Val Accuracy: 89.20000457763672
Iteration 500, Epoch 1, Loss: 0.38328954577445984, Accuracy: 88.80052185058594, Val Loss: 0.3398454487323761, Val Accuracy: 90.20000457763672
Iteration 600, Epoch 1, Loss: 0.3611729145050049, Accuracy: 89.41087341308594, Val Loss: 0.3246558904647827, Val Accuracy: 91.0
Iteration 700, Epoch 1, Loss: 0.34433794021606445, Accuracy: 8

Обучите трехслойную CNN. В tf.keras.optimizers.SGD укажите Nesterov momentum = 0.9 .

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/optimizers/SGD

Значение accuracy на валидационной выборке после 1 эпохи обучения должно быть > 50% .

In [10]:
learning_rate = 3e-3
channel_1, channel_2, num_classes = 32, 16, 10

def model_init_fn():
    model = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return model

def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9, nesterov=True)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.366486072540283, Accuracy: 10.9375, Val Loss: 2.342829465866089, Val Accuracy: 10.800000190734863
Iteration 100, Epoch 1, Loss: 0.6668420433998108, Accuracy: 80.75494384765625, Val Loss: 0.5221271514892578, Val Accuracy: 83.0
Iteration 200, Epoch 1, Loss: 0.5170328617095947, Accuracy: 85.07463073730469, Val Loss: 0.42704176902770996, Val Accuracy: 86.69999694824219
Iteration 300, Epoch 1, Loss: 0.4358997046947479, Accuracy: 87.41175842285156, Val Loss: 0.2882182002067566, Val Accuracy: 91.9000015258789
Iteration 400, Epoch 1, Loss: 0.37366825342178345, Accuracy: 89.14043426513672, Val Loss: 0.2394806444644928, Val Accuracy: 92.79999542236328
Iteration 500, Epoch 1, Loss: 0.3341791331768036, Accuracy: 90.3193588256836, Val Loss: 0.20790891349315643, Val Accuracy: 93.80000305175781
Iteration 600, Epoch 1, Loss: 0.3006870150566101, Accuracy: 91.2983627319336, Val Loss: 0.20222634077072144, Val Accuracy: 93.9000015258789
Iteration 700, Epoch 1, Loss: 0.2751112

# Использование Keras Sequential API для реализации последовательных моделей.

Пример для полносвязной сети:

In [13]:
learning_rate = 1e-2

def model_init_fn():
    input_shape = (28,28,1)
    hidden_layer_size, num_classes = 4000, 10
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    layers = [
        tf.keras.layers.Flatten(input_shape=input_shape),
        tf.keras.layers.Dense(hidden_layer_size, activation='relu',
                              kernel_initializer=initializer),
        tf.keras.layers.Dense(num_classes, activation='softmax',
                              kernel_initializer=initializer),
    ]
    model = tf.keras.Sequential(layers)
    return model

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 3.063081741333008, Accuracy: 14.0625, Val Loss: 2.650625467300415, Val Accuracy: 15.700000762939453
Iteration 100, Epoch 1, Loss: 0.6567699909210205, Accuracy: 79.99691009521484, Val Loss: 0.5153436660766602, Val Accuracy: 84.5999984741211
Iteration 200, Epoch 1, Loss: 0.5146311521530151, Accuracy: 84.6393051147461, Val Loss: 0.4504556357860565, Val Accuracy: 86.5
Iteration 300, Epoch 1, Loss: 0.4549024701118469, Accuracy: 86.48255920410156, Val Loss: 0.4057101309299469, Val Accuracy: 87.4000015258789
Iteration 400, Epoch 1, Loss: 0.4098382890224457, Accuracy: 87.87796020507812, Val Loss: 0.3639447093009949, Val Accuracy: 89.4000015258789
Iteration 500, Epoch 1, Loss: 0.3868507146835327, Accuracy: 88.59468841552734, Val Loss: 0.35020047426223755, Val Accuracy: 90.4000015258789
Iteration 600, Epoch 1, Loss: 0.3636132478713989, Accuracy: 89.27828979492188, Val Loss: 0.3281978368759155, Val Accuracy: 90.9000015258789
Iteration 700, Epoch 1, Loss: 0.345790773630

Альтернативный менее гибкий способ обучения:

In [14]:
model = model_init_fn()
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)

313/313 [==============================] - 3s 9ms/step - loss: 0.2238 - sparse_categorical_accuracy: 0.9355


[0.22382304072380066, 0.9355000257492065]

Перепишите реализацию трехслойной CNN с помощью tf.keras.Sequential API . Обучите модель двумя способами.

In [15]:
def model_init_fn():
    model = None
    ############################################################################
    # TODO: Construct a three-layer ConvNet using tf.keras.Sequential.         #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                            END OF YOUR CODE                              #
    ############################################################################
    return model

learning_rate = 5e-4
def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    optimizer = tf.keras.optimizers.SGD(learning_rate)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.2997660636901855, Accuracy: 9.375, Val Loss: 2.3207502365112305, Val Accuracy: 6.400000095367432
Iteration 100, Epoch 1, Loss: 2.290864944458008, Accuracy: 10.983911514282227, Val Loss: 2.2642266750335693, Val Accuracy: 16.200000762939453
Iteration 200, Epoch 1, Loss: 2.25793194770813, Accuracy: 18.275808334350586, Val Loss: 2.2039475440979004, Val Accuracy: 31.5
Iteration 300, Epoch 1, Loss: 2.224306344985962, Accuracy: 25.680023193359375, Val Loss: 2.1272246837615967, Val Accuracy: 45.89999771118164
Iteration 400, Epoch 1, Loss: 2.1791484355926514, Accuracy: 33.38138961791992, Val Loss: 2.0072836875915527, Val Accuracy: 58.0
Iteration 500, Epoch 1, Loss: 2.122676134109497, Accuracy: 39.670658111572266, Val Loss: 1.8313952684402466, Val Accuracy: 64.30000305175781
Iteration 600, Epoch 1, Loss: 2.0446672439575195, Accuracy: 44.922523498535156, Val Loss: 1.577453374862671, Val Accuracy: 68.80000305175781
Iteration 700, Epoch 1, Loss: 1.9474666118621826, Acc

In [16]:
model = model_init_fn()
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)

313/313 [==============================] - 3s 9ms/step - loss: 0.1717 - sparse_categorical_accuracy: 0.9471


[0.17168748378753662, 0.9470999836921692]

# Использование Keras Functional API

Для реализации более сложных архитектур сети с несколькими входами/выходами, повторным использованием слоев, "остаточными" связями (residual connections) необходимо явно указать входные и выходные тензоры.

Ниже представлен пример для полносвязной сети.

In [17]:
def two_layer_fc_functional(input_shape, hidden_size, num_classes):
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    inputs = tf.keras.Input(shape=input_shape)
    flattened_inputs = tf.keras.layers.Flatten()(inputs)
    fc1_output = tf.keras.layers.Dense(hidden_size, activation='relu',
                                 kernel_initializer=initializer)(flattened_inputs)
    scores = tf.keras.layers.Dense(num_classes, activation='softmax',
                             kernel_initializer=initializer)(fc1_output)

    # Instantiate the model given inputs and outputs.
    model = tf.keras.Model(inputs=inputs, outputs=scores)
    return model

def test_two_layer_fc_functional():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    input_shape = (50,)

    x = tf.zeros((64, input_size))
    model = two_layer_fc_functional(input_shape, hidden_size, num_classes)

    with tf.device(device):
        scores = model(x)
        print(scores.shape)

test_two_layer_fc_functional()

(64, 10)


In [20]:
input_shape = (28, 28, 1)
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn():
    return two_layer_fc_functional(input_shape, hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 3.1502909660339355, Accuracy: 6.25, Val Loss: 2.625807762145996, Val Accuracy: 15.80000114440918
Iteration 100, Epoch 1, Loss: 0.6259249448776245, Accuracy: 80.92512512207031, Val Loss: 0.5451792478561401, Val Accuracy: 82.5
Iteration 200, Epoch 1, Loss: 0.49515366554260254, Accuracy: 85.28451538085938, Val Loss: 0.44531258940696716, Val Accuracy: 87.19999694824219
Iteration 300, Epoch 1, Loss: 0.4393361806869507, Accuracy: 86.99647521972656, Val Loss: 0.40034258365631104, Val Accuracy: 87.30000305175781
Iteration 400, Epoch 1, Loss: 0.39786389470100403, Accuracy: 88.25592041015625, Val Loss: 0.3628551959991455, Val Accuracy: 89.30000305175781
Iteration 500, Epoch 1, Loss: 0.37650197744369507, Accuracy: 88.9252700805664, Val Loss: 0.3489495515823364, Val Accuracy: 90.5999984741211
Iteration 600, Epoch 1, Loss: 0.35414251685142517, Accuracy: 89.5668716430664, Val Loss: 0.33209818601608276, Val Accuracy: 90.9000015258789
Iteration 700, Epoch 1, Loss: 0.3374370

Поэкспериментируйте с архитектурой сверточной сети. Для вашего набора данных вам необходимо получить как минимум 70% accuracy на валидационной выборке за 10 эпох обучения. Опишите все эксперименты и сделайте выводы (без выполнения данного пункта работы приниматься не будут).

Эспериментируйте с архитектурой, гиперпараметрами, функцией потерь, регуляризацией, методом оптимизации.  

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/BatchNormalization#methods https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dropout#methods

In [25]:
class _IdentityBlock(tf.keras.Model):
    """Identity block utilizing skip connections."""

    def __init__(self, out_channels):
        super().__init__()
        """Initializes the identiy block.

        Here we simply initialize 2 layers which process the input and after the output
        is produces it is added together with the input whcih is the final output.

        Args:
            out_channels (int): The number of activation maps this block should produce
        """
        # Acts as Kaiming weight initalization
        initializer = tf.initializers.VarianceScaling(scale=2.0)

        # Part 1 of the convolution, normalization and non-linearity
        self.conv1 = tf.keras.layers.Conv2D(out_channels, 3, padding='same', use_bias=False, kernel_initializer=initializer)
        self.norm1 = tf.keras.layers.BatchNormalization(axis=3)
        self.relu1 = tf.keras.layers.Activation('relu')

        # Part 2 of the convolution, normalization and non-linearity
        self.conv2 = tf.keras.layers.Conv2D(out_channels, 3, padding='same', use_bias=False, kernel_initializer=initializer)
        self.norm2 = tf.keras.layers.BatchNormalization(axis=3)
        self.relu2 = tf.keras.layers.Activation('relu')

        # Add layer will add together the input and the output
        self.add = tf.keras.layers.Add()

    def call(self, x, training=False):
        """Performs forward pass on the given input.

        Args:
            x (Tensor):      The input of dimensions (N, H, W, C)
            training (bool): Indicates whether the forward pass happens in the training mode

        Returns:
              out (Tensor): Output data of dim (N, H, W, C)
        """
        x_skip = tf.identity(x)                   # prepare to add the input to the output
        x = self.relu1(self.norm1(self.conv1(x))) # pass input through the first layer
        x = self.norm2(self.conv2(x))             # pass input through the second layer (without ReLU)
        out = self.relu2(self.add([x, x_skip]))   # perform ReLU on the processed input added with the raw input

        return out


class ResNet(tf.keras.Model):

    def __init__(self, in_channels=32, block_config=(2, 2, 2, 2), num_classes=10):
        """Initializes the residual network.

        The first layer produces `in_channels` activation maps which are then fed to a
        sequence of blocks containing a specified number of identity sub-blocks (first
        block is always *_BottleneckBlock*). At the end the _global average pooling_
        layer is used to flatten the activations for the linear softmax classifier.

        Args:
            in_channels (int):    The number of channels to extract after the first convolution
            block_config (tuple): The number of layers each bloack should have in sequence
            num_classes (int):    The total number of classes
        """
        super().__init__()

        # Acts as Kaiming weight initalization
        initializer = tf.initializers.VarianceScaling(scale=2.0)

        # Prepare the input for the chains of identity blocks
        self.features = tf.keras.Sequential([
            tf.keras.layers.Conv2D(in_channels, 5, padding='same', use_bias=False, kernel_initializer=initializer),
            tf.keras.layers.BatchNormalization(axis=3),
            tf.keras.layers.Activation('relu'),
        ])

        num_features = in_channels # num feaure maps to produce after each group of identity blocks

        # Loop through every group of blocks
        for i, num_layers in enumerate(block_config):
            # Use bottleneck block as the first block in every group (except first)
            if i != 0:
                self.features.add(_BottleneckBlock(num_features))
            else:
                self.features.add(_IdentityBlock(num_features))

            # Create the specified number of identity blocks for i'th group
            for j in range(num_layers-1):
                self.features.add(_IdentityBlock(num_features))

            num_features *= 2 # increase the nuber of features to be produced

        # Flatten the final activation maps using global average pooling
        self.features.add(tf.keras.layers.GlobalAveragePooling2D())

        # Softmax classifier is used as the final layer
        self.classifier = tf.keras.layers.Dense(num_classes, activation='softmax', kernel_initializer=initializer)

    def call(self, x, training=False):
        """Performs forward pass on the given input.

        Args:
            x (Tensor):      The input of dimensions (N, H, W, C)
            training (bool): Indicates whether the forward pass happens in the training mode

        Returns:
            out (Tensor): Output data of dim (N, 10)
        """
        out = self.features(x)     # Get the extracted features for the linear classfier
        out = self.classifier(out) # Perform classification with softmax activation

        return out
class _BottleneckBlock(tf.keras.Model):
    """Same as identity block except it reduces the spacial area before processing the input."""

    def __init__(self, out_channels):
        """Initializes the bottleneck block.

        Unlike *_IdentityBlock*, the first convolution here reduces the spacial size of the input
        by a factor of `2`. Then, it performs the main convolution after which the output maps
        are added together with the input maps to produce final activations.

        Args:
            out_channels (int): The number of activation maps this block should produce
        """
        super().__init__()

        # Acts as Kaiming weight initalization
        initializer = tf.initializers.VarianceScaling(scale=2.0)

        # Reduce the input size by 2 to match output size
        self.skip1 = tf.keras.layers.Conv2D(out_channels, 1, strides=2, use_bias=False, kernel_initializer=initializer)

        # Part 1 of the convolution which reduces the spacial area
        self.conv1 = tf.keras.layers.Conv2D(out_channels, 3, strides=2, padding='same', use_bias=False, kernel_initializer=initializer)
        self.norm1 = tf.keras.layers.BatchNormalization(axis=3)
        self.relu1 = tf.keras.layers.Activation('relu')

        # Part 2 of the convolution which extracts features from the reduced input
        self.conv2 = tf.keras.layers.Conv2D(out_channels, 3, padding='same', use_bias=False, kernel_initializer=initializer)
        self.norm2 = tf.keras.layers.BatchNormalization(axis=3)
        self.relu2 = tf.keras.layers.Activation('relu')

        # Add layer will add together the input and the output
        self.add = tf.keras.layers.Add()

    def call(self, x, training=False):
        """Performs forward pass on the given input.

        Args:
            x (Tensor):      The input of dimensions (N, H, W, C)
            training (bool): Indicates whether the forward pass happens in the training mode

        Returns:
            out (Tensor): Output data of dim (N, H/2, W/2, out_channels)
        """
        x_skip = self.skip1(x)                    # prepare to add the input to the output
        x = self.relu1(self.norm1(self.conv1(x))) # pass input through the first layer
        x = self.norm2(self.conv2(x))             # pass input through the second layer (without ReLU)
        out = self.relu2(self.add([x, x_skip]))   # perform ReLU on the processed input added with the raw input

        return out


In [26]:
class CustomConvNet(tf.keras.Model):
    def __init__(self):
        super(CustomConvNet, self).__init__()
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        self.model = ResNet()

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################

    def call(self, input_tensor, training=False):
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        x = self.model.call(input_tensor, training)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
        return x


print_every = 100
num_epochs = 10

model = CustomConvNet()

def model_init_fn():
    return CustomConvNet()

def optimizer_init_fn():
    learning_rate = 1e-3
    return tf.keras.optimizers.Adam(learning_rate)

train_part34(model_init_fn, optimizer_init_fn, num_epochs=num_epochs, is_training=True)

Iteration 0, Epoch 1, Loss: 4.009794235229492, Accuracy: 6.25, Val Loss: 127.6778793334961, Val Accuracy: 10.699999809265137
Iteration 100, Epoch 1, Loss: 0.40878623723983765, Accuracy: 87.94863891601562, Val Loss: 5.104014873504639, Val Accuracy: 35.900001525878906
Iteration 200, Epoch 1, Loss: 0.2821394205093384, Accuracy: 91.66667175292969, Val Loss: 0.40873831510543823, Val Accuracy: 88.80000305175781
Iteration 300, Epoch 1, Loss: 0.22032441198825836, Accuracy: 93.48007202148438, Val Loss: 0.44638580083847046, Val Accuracy: 89.0
Iteration 400, Epoch 1, Loss: 0.182871475815773, Accuracy: 94.57605743408203, Val Loss: 0.4412424862384796, Val Accuracy: 88.30000305175781
Iteration 500, Epoch 1, Loss: 0.16443847119808197, Accuracy: 95.1316146850586, Val Loss: 0.22244659066200256, Val Accuracy: 93.0999984741211
Iteration 600, Epoch 1, Loss: 0.15051861107349396, Accuracy: 95.556884765625, Val Loss: 0.27871453762054443, Val Accuracy: 92.19999694824219
Iteration 700, Epoch 1, Loss: 0.1393807

Iteration 5900, Epoch 8, Loss: 0.01157704833894968, Accuracy: 99.60865020751953, Val Loss: 0.05232682451605797, Val Accuracy: 98.5999984741211
Iteration 6000, Epoch 8, Loss: 0.012547721154987812, Accuracy: 99.57942199707031, Val Loss: 0.11447490751743317, Val Accuracy: 97.5
Iteration 6100, Epoch 8, Loss: 0.013234215788543224, Accuracy: 99.5517578125, Val Loss: 0.11929888278245926, Val Accuracy: 97.5
Iteration 6200, Epoch 9, Loss: 0.01165104005485773, Accuracy: 99.5719223022461, Val Loss: 0.05349003151059151, Val Accuracy: 98.4000015258789
Iteration 6300, Epoch 9, Loss: 0.010218532755970955, Accuracy: 99.66582489013672, Val Loss: 0.05926158279180527, Val Accuracy: 98.4000015258789
Iteration 6400, Epoch 9, Loss: 0.009462381713092327, Accuracy: 99.70238494873047, Val Loss: 0.0628432184457779, Val Accuracy: 98.0999984741211
Iteration 6500, Epoch 9, Loss: 0.009630504995584488, Accuracy: 99.71932983398438, Val Loss: 0.05669794976711273, Val Accuracy: 98.9000015258789
Iteration 6600, Epoch 9,

Опишите все эксперименты, результаты. Сделайте выводы.

  Была реализована следующая архитектура:
        1. `CONV->NORM->RELU->CONV->NORM->RELU` для предварительной обработки входного сигнала для цепочки блоков слоев
        2. `IDENTITY->[BOTTLENECK->IDENTITY] x N`, где каждый блок состоит из произвольного
           где каждый блок состоит из произвольного количества слоев, использующих "skip" соединения
        3. `POOL->DENSE->SOFTMAX`, где выполняется глобальное усреднение пула
           перед вычислением raw scores
В результате произведенной работы были получены необходимые результаты уже на 3 эпохе